In [91]:
# libraries
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import json
import nltk
from nltk.corpus import stopwords
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import networkx as nx
import operator

file_path = 'C:\\Users\\aaami\\Desktop\\2022_1\\데이터애널리틱스\\과제\\8주차\\data_week8.json'
with open(file_path, 'r',encoding='utf-8') as f:
    json_data = json.load(f)
    
all_review_string = ''
titles = []
# review_body만 text mining data로 사용
for item in json_data:
    txt = item["review_body"].lower()   
    titles.append(item["review_head"])
    all_review_string += txt
    all_review_string += " "
    
# 스페이스, 구두점 단위로 키워드 분할 
word_tokens = nltk.word_tokenize(all_review_string)
# print(word_tokens)

tokens_pos = nltk.pos_tag(word_tokens)

# 그 중 명사만 가져오기
NN_words = []
for word, pos in tokens_pos:
    if 'NN' in pos:
        NN_words.append(word)
#print(NN_words)

# nltk에서 제공되는 WordNetLemmatizer을 이용
# ex) 명사의 경우는 보통 복수 -> 단수 형태로 변형
wlem = nltk.WordNetLemmatizer()
lemmatized_words = []
for word in NN_words:
    new_word = wlem.lemmatize(word)
    lemmatized_words.append(new_word)

# nltk에서 제공하는 불용어사전 이용
stopwords_list = stopwords.words('english') 
#print('stopwords: ', stopwords_list)
unique_NN_words = set(lemmatized_words)
final_NN_words = lemmatized_words

# 그 외에 제거되어야 할 단어를 stopwords로 등록
stopwords_list.append('dot')
stopwords_list.append('....')
stopwords_list.append('’')

# stopwords 제거
for word in unique_NN_words:
    if word in stopwords_list:
        while word in final_NN_words: final_NN_words.remove(word)
    

from collections import Counter
c = Counter(final_NN_words) # input type should be a list of words (or tokens)

# 가장 많이 나온 100개 단어만 추출
k = 100
most_common_words = []
i = 0
for word in c.most_common(k):
    most_common_words.append(word[0])
    
print(c.most_common(k))

# Document-term-matrix 생성
dtm = [[0 for col in range(len(most_common_words))] for row in range(len(docs))]

count = 0
for doc in docs:
    tmp = []
    for word in most_common_words:
        # 각 리뷰에서 키워드가 나타난 횟수
        tmp.append(doc.count(word))
    
    dtm[count] = tmp
    count+=1

# 테이블 생성 (기록용)
df = pd.DataFrame(dtm)
df.index = titles
df.columns = most_common_words
df.to_csv('C:\\Users\\aaami\\Desktop\\2022_1\\데이터애널리틱스\\과제\\8주차\\DTM.csv', encoding='utf-8-sig')

# Term corrlation matrix 생성
def dotOper(arr1, arr2):
    answer = np.dot(arr1, arr2).tolist()
    return answer
# 
tcm = dotOper(np.transpose(dtm), dtm)
    
# -------------------------- 키워드 동시출현 네트워크 생성 ----------------------------------

plt.figure(figsize=(50, 30))
df = pd.DataFrame(tcm, index=most_common_words, columns=most_common_words)
    
corr = df.corr()
 
links = corr.stack().reset_index()
links.columns = ['var1', 'var2', 'value']
 
links_filtered=links.loc[ (links['value'] > 0.5) & (links['var1'] != links['var2']) ]

G=nx.from_pandas_edgelist(links_filtered, 'var1', 'var2')
d = dict(G.degree)

nx.draw(G, nodelist = d.keys(), with_labels=True, node_color='cornflowerblue', node_size=[v *80 for v in d.values()], edge_color='black', linewidths=1, font_size=10)



[('music', 49), ('speaker', 38), ('alexa', 35), ('home', 31), ('sound', 31), ('device', 24), ('house', 24), ('echo', 23), ('room', 23), ('thing', 20), ('phone', 18), ('time', 18), ('weather', 16), ('app', 13), ('question', 12), ('show', 12), ('amazon', 12), ('purchase', 11), ('alarm', 11), ('list', 11), ('quality', 11), ('reminder', 10), ('news', 10), ('way', 10), ('lot', 9), ('light', 9), ('bedroom', 9), ('family', 8), ('product', 8), ('one', 8), ('hand', 8), ('voice', 8), ('volume', 8), ('skill', 8), ('sister', 8), ('joke', 7), ('feature', 7), ('son', 7), ('parent', 7), ('area', 6), ('buy', 6), ('life', 6), ('radio', 6), ('item', 6), ('intercom', 6), ('tv', 6), ('christmas', 6), ('gift', 5), ('night', 5), ('gen', 5), ('use', 5), ('problem', 5), ('play', 5), ('person', 5), ('siri', 5), ('brother', 5), ('day', 5), ('size', 5), ('generation', 5), ('everything', 5), ('timer', 5), ('information', 4), ('year', 4), ('friend', 4), ('anything', 4), ('version', 4), ('place', 4), ('system', 4),

NetworkXNotImplemented: not implemented for undirected type

<Figure size 3600x2160 with 0 Axes>